## News Source Attribution with FFNN

Necessary Imports

In [1]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
! pip install gensim
!pip install numpy==1.24.3 --force-reinstall
!pip install scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 54.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 11.8 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.14.1
    Uninstalling scipy-1.14.1:
      Successfully uninstalled scipy-1.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from nltk import FreqDist
import glob
from nltk.corpus import stopwords
import math
import re
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
import pandas as pd
# import numpy as np
import gensim
from gensim.models import Word2Vec

from sklearn import metrics


In [3]:
import numpy as np

In [4]:
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

Load Data

In [5]:

df = pd.read_csv('/content/all-the-news-2-1-SMALL-CLEANED.csv', encoding='latin-1')
print("Data loaded successfully.")

Data loaded successfully.


Load embedding model

In [8]:
!wget https://github.com/eyaler/word2vec-slim/raw/refs/heads/master/GoogleNews-vectors-negative300-SLIM.bin.gz

--2025-04-19 17:08:18--  https://github.com/eyaler/word2vec-slim/raw/refs/heads/master/GoogleNews-vectors-negative300-SLIM.bin.gz
Resolving github.com (github.com)... 140.82.116.4
Connecting to github.com (github.com)|140.82.116.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/eyaler/word2vec-slim/refs/heads/master/GoogleNews-vectors-negative300-SLIM.bin.gz [following]
--2025-04-19 17:08:18--  https://media.githubusercontent.com/media/eyaler/word2vec-slim/refs/heads/master/GoogleNews-vectors-negative300-SLIM.bin.gz
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 276467217 (264M) [application/octet-stream]
Saving to: ‘GoogleNews-vectors-negative300-SLIM.bin.gz’

GoogleNews-vecto

In [9]:
bigmodel = gensim.models.KeyedVectors.load_word2vec_format("GoogleNews-vectors-negative300-SLIM.bin.gz", binary=True)

Tokenize text by word

In [16]:

train_articles = []
test_articles = []
train_labels = []
test_labels = []
train_count = 0
test_count = 0
for index, row in df.iterrows():
    if row["split"] == "train":
      article = row["clean_article"]
      addme = [t.lower() for t in article.split()]# not in stoplist]
      train_articles.append(addme)
      train_labels.append(row["publication"])

    if row["split"] == "test":
      article = row["clean_article"]
      addme = [t.lower() for t in article.split()]# not in stoplist]
      test_articles.append(addme)
      test_labels.append(row["publication"])


In [10]:
def tokenize_text(text):
  totvec = np.zeros(300)
  for w in text:
      if w.lower() in bigmodel:
          totvec = totvec + bigmodel[w.lower()]
  totvec = totvec / len(article)
  return totvec

Build test + train data by getting average word embeddings of articles

In [11]:
train_vectors = []

for article in train_articles:
    totvec = np.zeros(300)
    for w in article:
        if w.lower() in bigmodel:
            totvec = totvec + bigmodel[w.lower()]
    totvec = totvec / len(article)
    train_vectors.append(totvec)


test_vectors = []
count = 0
for article in test_articles:

    totvec = np.zeros(300)
    for w in article:
        if w.lower() in bigmodel:
            totvec = totvec + bigmodel[w.lower()]
    totvec = totvec / len(article)
    test_vectors.append(totvec)


In [12]:
publications = df['publication'].unique()
print(publications)
publication_map = {publications[i]: i for i in range(len(publications))}
print(publication_map)

test_labels = [publication_map[p] for p in test_labels]
train_labels = [publication_map[p] for p in train_labels]

['The New York Times' 'The Hill' 'Reuters' 'People' 'CNN' 'Vice'
 'Politico' 'Buzzfeed News' 'Economist' 'Fox News']
{'The New York Times': 0, 'The Hill': 1, 'Reuters': 2, 'People': 3, 'CNN': 4, 'Vice': 5, 'Politico': 6, 'Buzzfeed News': 7, 'Economist': 8, 'Fox News': 9}


Run Neural Net

In [13]:
### NEURAL NET CLASSIFIER

print(f"\n\n\nLayers: 128,64,32 \n")

clf = MLPClassifier(solver='adam', alpha=1e-5, hidden_layer_sizes=(128, 64, 32), random_state=1)
clf.fit(train_vectors, train_labels)
predicted = clf.predict(test_vectors)
print("Accuracy of Multi-Layer Perceptron")
print(metrics.classification_report(test_labels, predicted))




#########################



Layers: (64, 32)



/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Accuracy of Multi-Layer Perceptron
              precision    recall  f1-score   support

           0       0.45      0.37      0.41      1000
           1       0.62      0.64      0.63      1000
           2       0.68      0.71      0.69      1000
           3       0.56      0.82      0.67      1000
           4       0.32      0.18      0.23      1000
           5       0.59      0.72      0.65      1000
           6       0.42      0.52      0.47      1000
           7       0.33      0.16      0.22      1000
           8       0.80      0.82      0.81      1000
           9       0.34      0.38      0.36      1000

    accuracy                           0.53     10000
   macro avg       0.51      0.53      0.51     10000
weighted avg       0.51      0.53      0.51     10000



#########################



Layers: (16, 8)



/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted s

Accuracy of Multi-Layer Perceptron
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1000
           1       0.21      0.05      0.08      1000
           2       0.28      0.79      0.41      1000
           3       0.41      0.61      0.49      1000
           4       0.06      0.01      0.02      1000
           5       0.21      0.31      0.25      1000
           6       0.21      0.01      0.01      1000
           7       0.16      0.21      0.18      1000
           8       0.19      0.47      0.27      1000
           9       0.00      0.00      0.00      1000

    accuracy                           0.25     10000
   macro avg       0.17      0.25      0.17     10000
weighted avg       0.17      0.25      0.17     10000



#########################



Layers: (128, 64)



/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Accuracy of Multi-Layer Perceptron
              precision    recall  f1-score   support

           0       0.29      0.27      0.28      1000
           1       0.49      0.63      0.55      1000
           2       0.60      0.65      0.62      1000
           3       0.57      0.76      0.65      1000
           4       0.24      0.07      0.11      1000
           5       0.53      0.71      0.61      1000
           6       0.38      0.39      0.39      1000
           7       0.33      0.18      0.24      1000
           8       0.66      0.88      0.75      1000
           9       0.33      0.26      0.29      1000

    accuracy                           0.48     10000
   macro avg       0.44      0.48      0.45     10000
weighted avg       0.44      0.48      0.45     10000



#########################



Layers: (20, 10)



/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:698: UserWarning: Training interrupted by user.
  warnings.warn("Training interrupted by user.")


Accuracy of Multi-Layer Perceptron
              precision    recall  f1-score   support

           0       0.42      0.38      0.40      1000
           1       0.60      0.71      0.65      1000
           2       0.62      0.74      0.67      1000
           3       0.63      0.73      0.68      1000
           4       0.19      0.07      0.10      1000
           5       0.57      0.75      0.65      1000
           6       0.44      0.52      0.48      1000
           7       0.29      0.10      0.15      1000
           8       0.75      0.88      0.81      1000
           9       0.33      0.38      0.35      1000

    accuracy                           0.53     10000
   macro avg       0.48      0.53      0.49     10000
weighted avg       0.48      0.53      0.49     10000



#########################



Layers: [128, 64, 32]

Accuracy of Multi-Layer Perceptron
              precision    recall  f1-score   support

           0       0.69      0.70      0.70      1000
        

/usr/local/lib/python3.11/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
